In [ ]:

#!pip install -q torchmetrics # trên kaggle có lib này rồi, không cần install cũng dc
!pip install -q segmentation-models-pytorch

import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.train import TrainEpoch as SMPTrainEpoch

# --- DATASET ---
class CloudDataset(BaseDataset):
    def __init__(self, images_dir, masks_dir=None, augmentation=None, preprocessing=None, has_masks=True):
        # Lọc chỉ lấy file _sat.jpg để tránh lấy nhầm file khác
        self.ids = [os.path.splitext(f)[0].replace('_sat', '') for f in os.listdir(images_dir) if f.endswith('_sat.jpg')]
        self.images_fps = [os.path.join(images_dir, image_id + '_sat.jpg') for image_id in self.ids]
        self.has_masks = has_masks
        
        # Đường dẫn mask (Nếu dataset chuẩn DeepGlobe thì đuôi là _mask.png)
        if self.has_masks:
            self.masks_fps = [os.path.join(images_dir, image_id + '_mask.png') for image_id in self.ids]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, i):
        # Đọc ảnh
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (512, 512))

        mask_forest = None
        if self.has_masks:
            mask = cv2.imread(self.masks_fps[i])
            if mask is None:
                # Fallback nếu không đọc được mask
                mask_forest = np.zeros((512, 512, 1), dtype=np.float32)
            else:
                mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
                mask = cv2.resize(mask, (512, 512))
                
                # Mask màu xanh lá (Forest)
                lower_green = np.array([0, 250, 0])
                upper_green = np.array([5, 255, 5])
                mask_forest = cv2.inRange(mask, lower_green, upper_green)
                
                # Chuẩn hóa về 0 và 1
                mask_forest = (mask_forest / 255.0).astype('float32')
                mask_forest = np.expand_dims(mask_forest, axis=-1)
        else:
            mask_forest = np.zeros((512, 512, 1), dtype=np.float32)

        # Augmentation (nếu có)
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask_forest)
            image, mask_forest = sample['image'], sample['mask']

        # Preprocessing (Chuẩn hóa theo Imagenet)
        if self.preprocessing:
            image = self.preprocessing(image)

        # Chuyển về Tensor (HWC -> CHW)
        image = torch.from_numpy(image.transpose(2, 0, 1)).float()
        mask_forest = torch.from_numpy(mask_forest.transpose(2, 0, 1)).float()

        return image, mask_forest

    def __len__(self):
        return len(self.images_fps)

# --- CONFIG ---
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Tạo Model
model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=1,
    activation='sigmoid', # Output là xác suất (0-1)
).to(DEVICE)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

# --- PATHS ---
# Đảm bảo folder này tồn tại bên cột phải màn hình Kaggle
x_train_dir = "/kaggle/input/deepglobe-land-cover-classification-dataset/train"

# Check xem đường dẫn có đúng không trước khi chạy
if not os.path.exists(x_train_dir):
    print(f"CẢNH BÁO: Đường dẫn {x_train_dir} không tồn tại!")
    print("Kiểm tra lại tên dataset đã Add vào ")
else:
    print("Đường dẫn hợp lệ.")

# Tạo Dataset & Loader
train_dataset = CloudDataset(
    x_train_dir, 
    preprocessing=preprocessing_fn,
    has_masks=True
)

# GPU P100 RAM 16GB, Batch 10-16 là đẹp cho ResNet34
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

# --- LOSS & METRICS ---
# Quan trọng: from_logits=False vì model đã có sigmoid
loss = smp.losses.DiceLoss(mode='binary', from_logits=False) 
loss.__name__ = 'dice_loss'

# Dùng Metric nội bộ của SMP để tránh lỗi tương thích
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

# Tạo Epoch Runner
train_epoch = SMPTrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

# --- TRAINING LOOP ---
print(f"Bắt đầu train trên thiết bị: {DEVICE}")

# Save path
save_path = '/kaggle/working/unet_forest_segmentation_model.pth'

best_iou = 0.0

for i in range(20):
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    
    # Logic lưu model: Chỉ lưu nếu IoU tăng (tùy chọn) hoặc lưu đè mỗi epoch
    # Ở đây mình lưu đè mỗi epoch cho đơn giản
    try:
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")
    except Exception as e:
        print(f"Error saving model: {e}")

: 